In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import openai
from dotenv import load_dotenv

from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs

from method.ours.utils import create_driver
from method.ours.parse import (
    initial_parse,
    parse_after_feedback,
)

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Global Variables
HEADLESS = False
TEXT_EMBEDDING_METHOD = 'ADA' # ['ADA', 'WORD2VEC', 'SPACY']
GRAPH_EMBEDDING_METHOD = 'NODE2VEC' # ['NODE2VEC', 'GCN']

In [4]:
driver = create_driver(HEADLESS)
driver.get('https://www.aircanada.com/ca/en/aco/home.html')

In [5]:
# form = driver.find_elements(By.TAG_NAME, 'form')[48]
# form = driver.find_element(By.ID, 'register')
form = driver.find_elements(By.TAG_NAME, 'form')[1]

# Processing

In [6]:
relation_graph, input_groups = initial_parse(driver, form, TEXT_EMBEDDING_METHOD)

Computing transition probabilities:   0%|          | 0/340 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:09<00:00,  3.93it/s]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openai/embeddings_utils.py:68: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


# Generation

In [7]:
from method.ours.constraints import generate_constraints_from_string
from method.ours.prompts import (
    create_constraint_generation_user_prompt,
    create_value_generation_user_prompt,
)
from method.ours.generation import (
    generate_constraints_with_llm,
    generate_values_with_llm,
)

In [8]:
class FillTable:
    def __init__(self):
        self.entries = {}
    
    
    def add_entry(self, field_id, input_group, constraints):
        self.entries[field_id] = FillTableRow(input_group, constraints)
    
    
    def get_entry_by_field_id(self, field_id):
        return self.entries[field_id]
    
    
    def get_entry_by_input_group(self, input_group):
        filtered = list(filter(
            lambda x: x.input_group.node.xpath == input_group.node.xpath,
            self.entries.values()
        ))
        if len(filtered) > 0:
            return filtered[0]
        return None


class FillTableRow:
    def __init__(self, input_group, constraints):
        self.input_group = input_group
        self.constraints = constraints
        self.feedback = None
        self.value = None
    
    
    def set_value(self, value):
        self.value = value
    
    
    def set_feedback(self, feedback):
        self.feedback = feedback

In [9]:
fill_table = FillTable()

In [10]:
for input_group in input_groups:
    # TODO: set more conditions on <button> to be sure that buttons here are inputs
    if input_group.node.element.name == 'button':
        continue
    
    constraint_user_prompt = create_constraint_generation_user_prompt(input_group)
    
    generated_constraints = generate_constraints_with_llm(
        constraint_user_prompt,
        openai_api_key=openai.api_key
    )
    
    field_name, constraints = generate_constraints_from_string(generated_constraints)
    
    fill_table.add_entry(field_name, input_group, constraints)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a set of constraints for form fields. Your decisions must be made independently without seeking user assistance or additional information.\n\nThe list of the constraints and their signatures is as follows:\n\n1. toBe(value) # the input field value exactly matches the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)\n4. toHaveCompareCondition(condition, value) # the input field value has the given condition to the given value\n5. toContainSubstr(value) # the input field value contains a specific string\n6. toContainChar(value) # the input field value contains a specific character\n7. toBeAlpha() # the input field value is alphabetic\n8. toBeNumeric() # the input field value is numeric\n9. toBeAlphaNumeric() # the input value conta

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('tripType'))\n.toBe('M')",
        "role": "assistant"
      }
    }
  ],
  "created": 1687300747,
  "id": "chatcmpl-7TeI7vUwZZJnlkjPgJFXXKOG9kjfE",
  "model": "gpt-4-0314",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 650,
    "total_tokens": 660
  }
}
  Total running cost: $0.060
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a set of constraints for form fields. Your decisions must be made independently without seeking user assistance or additional information.\n\nThe list of the constraints and their signatures is as follows:\n\n1. toBe(value) # the input field value exactly matches the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empt

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_destination_trip_1'))\n.toBeAlpha()\n.not.toBe(field('bkmgFlights_origin_trip_1'))\n.not.toHaveWhiteSpace()",
        "role": "assistant"
      }
    }
  ],
  "created": 1687300757,
  "id": "chatcmpl-7TeIHxfWwzq6TB71SV5rpblI2I5PI",
  "model": "gpt-4-0314",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 33,
    "prompt_tokens": 678,
    "total_tokens": 711
  }
}
  Total running cost: $0.125
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a set of constraints for form fields. Your decisions must be made independently without seeking user assistance or additional information.\n\nThe list of the constraints and their signatures is as follows:\n\n1. toBe(value) # the input field value exactly matches the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field v

In [11]:
def split_constant_and_field_constraints(constraints):
    constants = list(filter(lambda x: not x.is_field, constraints))
    fields = list(filter(lambda x: x.is_field, constraints))
    return constants, fields

In [14]:
# value generation loop

for input_group in input_groups:
    if input_group.node.element.name == 'button':
        continue
    
    fill_entry = fill_table.get_entry_by_input_group(input_group)
    
    if fill_entry is None:
        continue
    
    constraints = fill_entry.constraints
    constant_constraints, field_constraints = split_constant_and_field_constraints(constraints)
    including_constraints = [*constant_constraints]

    relevant_field_values = []
    for field_constraint in field_constraints:
        field_arg = field_constraint.get_field_args()[0]
        value = fill_table.get_entry_by_field_id(field_arg).value
        
        if value is not None:
            including_constraints.append(field_constraint)
            relevant_field_values.append((field_arg, value))
    
    value_user_prompt = create_value_generation_user_prompt(
        fill_entry.input_group,
        including_constraints,
        relevant_field_values=relevant_field_values if len(relevant_field_values) > 0 else None,
    )
    
    generated_value = generate_values_with_llm(
        value_user_prompt,
        openai_api_key=openai.api_key
    )
    
    fill_entry.set_value(generated_value)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a form field based on the form field information and a set of constraints on the field. Your decisions must always be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfy the constraints while keeping in mind the nature of the input from the available information.\nOnly generate the value and don't explain your answers.\nOnly generate value for the inputs in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_tripTypeSelector_M\" name=\"tripType\" type=\"radio\" value=\"M\"/>\nwith label: Multi-city/Stopover\n    The relevant information available in the form are (in order of relevance):\n    1.\ninput: <input id=\"bkmgFlights_tripTypeSelector_O\" nam

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "15/09",
        "role": "assistant"
      }
    }
  ],
  "created": 1687300819,
  "id": "chatcmpl-7TeJHhlMCSypSE7plx94IVDuw9D5t",
  "model": "gpt-4-0314",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 3,
    "prompt_tokens": 372,
    "total_tokens": 375
  }
}
  Total running cost: $0.264
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a form field based on the form field information and a set of constraints on the field. Your decisions must always be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfy the constraints while keeping in mind the nature of the input from the available information.\nOnly generate the value and don't explain your answers.\nOnly generate value for the inputs in question, and not the

In [16]:
for input_group in input_groups:
    if input_group.node.element.name == 'button':
        continue
    
    entry = fill_table.get_entry_by_input_group(input_group)
    
    if entry is None:
        continue
    
    print(entry.input_group.node)
    print(entry.value)

<input>value is M</input> at y: (567, 597), x: (437, 591)
M
<input></input> at y: (574, 594), x: (652, 905)
true
<input></input> at y: (628, 678), x: (198, 466)
Vancouver International Airport
<input></input> at y: (628, 678), x: (536, 804)
Toronto Pearson Airport
<input>DD/MM</input> at y: (628, 678), x: (833, 958)
15/09
<input>DD/MM</input> at y: (628, 678), x: (978, 1093)
20/09


# Feedback

In [8]:
new_form = driver.find_elements(By.TAG_NAME, 'form')[1]

In [9]:
new_relation_graph, new_input_groups = parse_after_feedback(driver, new_form, relation_graph, TEXT_EMBEDDING_METHOD)

Computing transition probabilities:   0%|          | 0/344 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [00:09<00:00,  4.02it/s]


{'added': [<span-wrap> Round-trip </span-wrap> at y: (572, 592), x: (249, 319), <span-wrap> One-way </span-wrap> at y: (572, 592), x: (364, 421), <span-wrap> Multi-city/Stopover </span-wrap> at y: (572, 592), x: (467, 591), <span-wrap> Book with points </span-wrap> at y: (574, 594), x: (682, 905), <div>Please select a valid destination for this trip.</div> at y: (685, 701), x: (536, 781), <div>Please select valid travel dates for this trip.</div> at y: (685, 701), x: (833, 1071), <span-wrap> Enter the date day and month in this format: DD/MM, or use the 'Show Calendar' button to open the calendar and select your date from there. </span-wrap> at y: (705, 706), x: (832, 833), <span-wrap> Add promotion code </span-wrap> at y: (743, 763), x: (200, 379), <span-wrap> Search flights </span-wrap> at y: (742, 762), x: (1202, 1306)], 'removed': [<span-wrap> Enter the date day and month in this format: DD/MM, or use the 'Show Calendar' button to open the calendar and select your date from there. 

In [10]:
new_relation_graph.edges()

[edge FOR weight 1 from <label> Round-trip </label> at y: (572, 592), x: (249, 319) to <input>value is R</input> at y: (567, 597), x: (219, 319),
 edge FOR weight 1 from <input>value is R</input> at y: (567, 597), x: (219, 319) to <label> Round-trip </label> at y: (572, 592), x: (249, 319),
 edge FOR weight 1 from <label> One-way </label> at y: (572, 592), x: (364, 421) to <input>value is O</input> at y: (567, 597), x: (334, 421),
 edge FOR weight 1 from <input>value is O</input> at y: (567, 597), x: (334, 421) to <label> One-way </label> at y: (572, 592), x: (364, 421),
 edge FOR weight 1 from <label> Multi-city/Stopover </label> at y: (572, 592), x: (467, 591) to <input>value is M</input> at y: (567, 597), x: (437, 591),
 edge FOR weight 1 from <input>value is M</input> at y: (567, 597), x: (437, 591) to <label> Multi-city/Stopover </label> at y: (572, 592), x: (467, 591),
 edge FOR weight 1 from <label> Book with points </label> at y: (574, 594), x: (682, 905) to <input></input> at 